In [1]:
import os
import pyspark 
from pyspark.sql import SparkSession


In [2]:
from dotenv import load_dotenv
load_dotenv()
key_filepath = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

In [3]:
spark = SparkSession.builder \
    .master("local[3]") \
    .appName("Airbnb Listings") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile",key_filepath  ) \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .getOrCreate()       


    

In [4]:

# Read data from GCS
df = spark.read.parquet("gs://airbnb-listings-421017-bucket/airbnb_listings.parquet")


In [13]:
df.show(5)

+-------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+----------------+---------------------+----------------------+--------+---------+----------+------------------+--------------------+-------+--------------------+-------------+---------------+------------+---------+--------+----+-----------------+--------------------+-------------------+-----------------+-----+-----------------+
|     id|                name|             summary|          longitude|          latitude|               space|         description|instant_bookable|neighborhood_overview|neighbourhood_cleansed| host_id|host_name|host_since|host_response_time|              street|zipcode|review_scores_rating|property_type|      room_type|accommodates|bathrooms|bedrooms|beds|reviews_per_month|           amenities|cancellation_policy|number_of_reviews|price|__index_level_0__|
+-------+--------------------+--------------------+-------------------+-----

In [7]:
# Check if df is an RDD
if isinstance(df, pyspark.rdd.RDD):
    print("df is an RDD")
else:
    print("df is not an RDD")


df is not an RDD


In [10]:
# Parse DataFrame into (X, y) pairs
def parse_point(row):
    features = [float(row[column]) for column in row.asDict().keys()[:-1]]
    return (features, float(row[-1]))

# Apply parse_point function to each row and select relevant columns
parsed_data = df.rdd.map(parse_point)

In [14]:
# Initialize coefficients (weights)
print(parsed_data.first())
# num_features = len(parsed_data.first()[0])
# weights = np.random.rand(num_features)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6) (Lenovo-Ideapad-520 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "E:\spark-3.5.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.10, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at or

In [ ]:
print(len(num_features))

In [ ]:
# Define the Hypothesis Function
def hypothesis(x, weights):
    return np.dot(x, weights)

In [ ]:
# Define the Cost Function
def cost_function(data, weights):
    n = data.count()
    squared_errors = data.map(lambda point: (hypothesis(point[0], weights) - point[1]) ** 2)
    return squared_errors.reduce(lambda x, y: x + y) / (2 * n)


In [ ]:
# Gradient Descent
def gradient_descent(data, weights, learning_rate):
    n = data.count()
    gradients = data.map(lambda point: np.multiply(hypothesis(point[0], weights) - point[1], point[0]))
    gradient_sum = gradients.reduce(lambda x, y: np.add(x, y))
    return np.subtract(weights, np.multiply(learning_rate / n, gradient_sum))


In [ ]:
# Hyperparameters
learning_rate = 0.01
num_iterations = 100

In [ ]:
# Training loop
for _ in range(num_iterations):
    weights = gradient_descent(parsed_data, weights, learning_rate)
    cost = cost_function(parsed_data, weights)
    print("Cost:", cost)


In [ ]:
# Stop SparkSession
spark.stop()